# Toronto Neighborhoods
## Import libraries

In [49]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import folium

print("Libraries imported!")

Libraries imported!


## Setting up Data
The data is scraped from [this Wikipedia Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Postal codes with no boroughs assigned are dropped and the column names and contents are modified into the desired format.

In [50]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res)
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

df.rename(columns={'Postal code':'PostalCode'},inplace=True)

df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)

df.Neighborhood = df.Neighborhood.str.replace(" /",",")

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [51]:
df.shape

(103, 3)

## Importing Latitude and Longitudes
Using the geocoder package was unsuccessful, so we will just download the CSV file and merge it with our original dataframe.

In [52]:
lat_lng = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng.rename(columns={'Postal Code':'PostalCode'},inplace=True)
lat_lng.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df = df.merge(lat_lng)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [54]:
df.shape

(103, 5)

## Neighborhood Analysis
### Restrict Data
Let's restrict our data to only those that contain Toronto in the name of the Borough

In [62]:
df = df[df.Borough.str.contains("Toronto")]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [63]:
df.shape

(39, 5)

### Visualizing the Map
Let's create a folium map object with markers for each postal code that have popups with the names of the neighborhoods within them.

*Note: Folium Maps will not render properly in GitHub. Follow the link [here](https://nbviewer.jupyter.org/github/alexanderWhile/Coursera_Capstone/blob/master/toronto_neighborhoods.ipynbj) to see the full output*

In [60]:
m = folium.Map(
    location=[43.6532, -79.3832],
    zoom_start=11)

for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Latitude'],df.iloc[i]['Longitude']],
        popup=df.iloc[i]['Neighborhood']
        ).add_to(m)

m